Märkmik ingliskeelsete PDF-ide parsimiseks teegi PyMUPDF abil ning sisuliselt sidusate lõikude eraldamiseks vektorandmebaasis säilitamise eesmärgil.

Sammud:
* Metaandmete säilitamine?
* Päiste eemaldamine (reguluaaravaldisega vs koordinaatidega vms?)
* Tabelite leidmine tekstist
* Tabelite eemaldamine tekstist
* Tükeldamine

Kas footnotes'id saab eristada muust tekstist (tabeli määratluse kaudu??)
Kui lehekülg ei lõpe kirjavahemärgiga, siis liita tekstid kokku?



In [ ]:
import fitz
import os
from collections import defaultdict

from parser_utils import get_all_tables, print_n_pages, calculate_content_box, is_inside_bbox

In [ ]:
# Muudetavad parameetrid

fdir = 'jp_data'
fname = os.listdir(fdir)[0]

table_extraction_strategy = 'lines_strict' # Strateegiad kirjeldatud: https://pymupdf.readthedocs.io/en/latest/page.html#Page.find_tables, saab valida failipõhiselt sobiva strateegia.
horizontal_sorting = True # Määrata False, kui tegemist on veergudepõhise dokumendiga.

header_height = 50
footer_height = 50

---
10 lk või bloki kuvamine näitena. Saab välja võtta päiste ja jaluste koordinaadid ning need muudetavatesse parameetritesse kopeerida, et saaks neid lõike ignoreerida.
Koordinaadid on blocki 4 esimest elementi.

In [ ]:
print_n_pages(fdir, fname, display='blocks')

In [ ]:
print_n_pages(fdir, fname, n = 10, display='text')

---
### Teksti ja tabelite eraldamine

Tabelite eemaldamine tekstist. Kogu teksti väljatrükk

In [ ]:
document_path = os.path.join(fdir, fname)

with fitz.open(document_path) as doc:

    document_tables = get_all_tables(doc, header_height, footer_height, table_extraction_strategy=table_extraction_strategy) # Tabeleid eraldatakse päise ja jaluseta piirkonnast, et päise eraldusjooned ei tekitaks liigseid tabeleid

    non_table_blocks = defaultdict(list)

    for page_no, page in enumerate(doc):
        
        # Teksti eraldamine päist ja jalust välja jättes
        text_blocks = page.get_text('blocks', sort=horizontal_sorting, clip=calculate_content_box(page, header_height, footer_height))

        # Teksti lisamine non_table_blocks muutujasse, ei lisata teksti, mis kuulub tabelisse
        for text_block in text_blocks:
            block_is_table_data = False
            for table in document_tables[page_no]:
                if is_inside_bbox(text_block, table.attrs['bbox']):
                    block_is_table_data = True
                    break
            if not block_is_table_data:
                non_table_blocks[page_no].append(text_block[4])

    # Puhastatud teksti koostamine
    cleaned_text = "\n".join(block for blocks in non_table_blocks.values() for block in blocks)
    print(cleaned_text)

2. Tabelite väljatrükk

In [ ]:
i = 20

for page, tables in document_tables.items():
    for table in tables:
        print('Page: {}'.format(table.attrs['page']))
        display(table)
    i -= 1
    if i == 0:
        break